In [5]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
%pip install tensorflow 
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model

     |████████████████████████████████| 165.2 MB 176 kB/s eta 0:00:01
     |████████████████████████████████| 459 kB 711 kB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 28.8 MB 750 kB/s eta 0:00:01
     |████████████████████████████████| 42 kB 447 kB/s eta 0:00:01
     |████████████████████████████████| 6.8 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 2.3 

In [6]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
print(ratings.shape)

(100836, 4)


In [7]:
train, test = train_test_split(ratings, test_size=0.2, random_state=6)

In [8]:
n_users = ratings.userId.nunique()
print('n_users =', n_users)

n_users = 610


In [9]:
n_movies = ratings.movieId.nunique()
print('n_movies =', n_movies)

n_movies = 9724


In [36]:
# creating movie embedding path
movie_input = Input(shape=[1], name="Movie-Input")
movie_embedding = Embedding(n_movies + 1, 5, name="Movie-Embedding")(movie_input)
movie_vec = Flatten(name="Flatten-Movies")(movie_embedding)

In [37]:
# creating user embedding path
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users + 1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

In [38]:
# concatenate features
conc = Concatenate()([movie_vec, user_vec])

In [39]:
# add fully-connected-layers
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

In [40]:
# Create model and compile it
model = Model([user_input, movie_input], out)
print(model.summary())
print('')
model.compile('adam', 'mean_squared_error')

Model: "functional_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Movie-Input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
User-Input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
Movie-Embedding (Embedding)     (None, 1, 5)         48625       Movie-Input[0][0]                
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 5)         3055        User-Input[0][0]                 
_______________________________________________________________________________________

In [43]:
history = model.fit([train.userId, train.movieId], train.rating, epochs=5, verbose=True)
model.save('MovieLensModel')
plt.plot(history.history['loss'])
plt.xlabel("Epochs")
plt.ylabel("Training Error")

Epoch 1/5


InvalidArgumentError:  indices[1,0] = 83134 is not in [0, 9725)
	 [[node functional_8/Movie-Embedding/embedding_lookup (defined at <ipython-input-41-6a660c615792>:1) ]] [Op:__inference_train_function_3328]

Errors may have originated from an input operation.
Input Source operations connected to node functional_8/Movie-Embedding/embedding_lookup:
 functional_8/Movie-Embedding/embedding_lookup/3064 (defined at /Users/sebastianbudd/opt/anaconda3/lib/python3.8/contextlib.py:113)

Function call stack:
train_function


In [44]:
predictions = model.predict([test.userId.head(10), test.movieId.head(10)])

InvalidArgumentError:  indices[5,0] = 45499 is not in [0, 9725)
	 [[node functional_8/Movie-Embedding/embedding_lookup (defined at <ipython-input-44-fc8090266d6e>:1) ]] [Op:__inference_predict_function_3575]

Errors may have originated from an input operation.
Input Source operations connected to node functional_8/Movie-Embedding/embedding_lookup:
 functional_8/Movie-Embedding/embedding_lookup/3541 (defined at /Users/sebastianbudd/opt/anaconda3/lib/python3.8/contextlib.py:113)

Function call stack:
predict_function


In [45]:
[print(predictions[i], test.rating.iloc[i]) for i in range(0, 10)]

NameError: name 'predictions' is not defined